# Tutorial 1.1: Working with the Lisa cluster

No actual coding, this is a purely markdown notebook

## First steps

### How to connect to Lisa

```bash
ssh -X lgqu____@lisa.surfsara.nl
```

### Modules

```bash
module purge
module load 2019
module load Python/3.7.5-foss-2019b
module load CUDA/10.1.243
module load cuDNN/7.6.5.32-CUDA-10.1.243
module load NCCL/2.5.6-CUDA-10.1.243
```

### Install the environment

### Template for a job file

```bash
#!/bin/bash

#SBATCH --partition=gpu_shared_course
#SBATCH --gres=gpu:1
#SBATCH --job-name=ExampleJob
#SBATCH --cpus-per-task=3
#SBATCH --ntasks=1
#SBATCH --time=04:00:00
#SBATCH --mem=32000M
#SBATCH --output=slurm_output_%A.out

module purge
module load 2019
module load Python/3.7.5-foss-2019b
module load CUDA/10.1.243
module load cuDNN/7.6.5.32-CUDA-10.1.243
module load NCCL/2.5.6-CUDA-10.1.243

cd ...
source activate ...
srun python -u ...
```

## The SLURM scheduler

### Queueing

### Organizing your jobs

## Troubleshooting

### Lisa is refusing connection

### Slurm output file missing